# Double bottleneck simulation 

In this demonstration, we'll construct a **double bottleneck** situation and simulate the evacuation of **agents** positioned on a grid.

**TODO: MORE EXPLANATION. What happens here? Why? Why do we hope to show?**

Introduction[CB]

The simulation of the double bottleneck is based on the work for the (single) bottleneck. 
Here, agents are positioned on both side of the bottleneck: left and right.
How will the agents "meet"? 
Will they disturb each other? Is it even possible to go through? 
And what is the impact of the width of the bottleneck?In order to answer those questions, please follow the instructions below. 

Let's begin by importing the required packages for our simulation:

In [ ]:
from shapely import GeometryCollection, Polygon, to_wkt
import pathlib
import pandas as pd
import numpy as np
import jupedsim as jps
from jupedsim.distributions import distribute_by_number
import sqlite3  # parse trajectory db
import plotly.express as px
import plotly.graph_objects as go
from plotly.graph_objs import Figure
import pedpy  # analysis

%matplotlib inline

## Setting up a geometry

In [ ]:
p1 = Polygon([(0, 0), (10, 0), (10, 10), (0, 10)])
p2 = Polygon([(10, 4.5), (15, 4.5), (15, 5.5), (10, 5.5)])
p3 = Polygon([(15, 0), (25, 0), (25, 10), (15, 10)])
area = GeometryCollection(p1.union(p2.union(p3)))
walkable_area = pedpy.WalkableArea(area.geoms[0])
pedpy.plot_walkable_area(walkable_area=walkable_area)

## Operational model
<a id="model"></a>
Once the geometry is set, our subsequent task is to specify the model and its associated parameters.
For this demonstration, we'll employ the "collision-free" model.

## Setting Up the Simulation Object

Having established the model and geometry details, and combined with other parameters such as the time step dt, we can proceed to construct our simulation object as illustrated below:

In [ ]:
trajectory_file = "double_bottleneck_1m_width.sqlite"
simulation = jps.Simulation(
    model=jps.VelocityModelParameters(),
    geometry=area,
    trajectory_writer=jps.SqliteTrajectoryWriter(
        output_file=pathlib.Path(trajectory_file)
    ),
)

## Specifying Routing Details

At this juncture, we'll provide basic routing instructions, guiding the agents to progress towards the **first exit point** and the **second exit point**.

In [ ]:
exits = [
    simulation.add_exit_stage([(24, 0), (25, 0), (25, 10), (24, 10)]),
    simulation.add_exit_stage([(0, 0), (1, 0), (1, 10), (0, 10)]),
]
journeys = [
    simulation.add_journey(jps.JourneyDescription([exit])) for exit in exits
]

## Defining and Distributing Agents

Now, we'll position the agents and establish their attributes, leveraging previously mentioned parameters such as `exit1`, `exit2` and `profile_id`.

In [ ]:
agent_parameters = jps.VelocityModelAgentParameters()
agent_parameters.journey_id = journeys[0]
agent_parameters.stage_id = exits[0]
agent_parameters.orientation = (1.0, 0.0)
agent_parameters.v0 = 1.2
agent_parameters.radius = 0.15
agent_parameters.time_gap = 1

for new_pos in [(7, 7)]:
    agent_parameters.position = new_pos
    simulation.add_agent(agent_parameters)

In [ ]:
agent_parameters.journey_id = journeys[1]
agent_parameters.stage_id = exits[1]
for new_pos in [(25, 7)]:
    agent_parameters.position = new_pos
    simulation.add_agent(agent_parameters)

## Executing the Simulation

With all components in place, we're set to initiate the simulation.
For this demonstration, the trajectories will be recorded in an sqlite database.

In [ ]:
while simulation.agent_count() > 0:
    simulation.iterate()

## Visualizing the Trajectories

For trajectory visualization, we'll extract data from the sqlite database. A straightforward method for this is employing the jupedsim-visualizer.

To-Do List:

    Incorporate references and hyperlinks to additional resources.
    Integrate results visualization using the visualizer.

In [ ]:
def read_trajectory_data(trajectory_file: str) -> pedpy.TrajectoryData:
    with sqlite3.connect(trajectory_file) as con:
        data = pd.read_sql_query(
            "select frame, id, pos_x as x, pos_y as y, ori_x as ox, ori_y as oy from trajectory_data",
            con,
        )
        fps = float(
            con.cursor()
            .execute("select value from metadata where key = 'fps'")
            .fetchone()[0]
        )
        return pedpy.TrajectoryData(data=data, frame_rate=fps)


def visualize(data_df: pd.DataFrame, area: GeometryCollection) -> Figure:
    """Generate moving trajectories based on simulation"""
    range_color = [0, max(data_df["speed"])]
    data_df["radius"] = 0.2
    fig = px.scatter(
        data_df,
        x="x",
        y="y",
        animation_frame="frame",
        animation_group="id",
        size="radius",
        color="speed",
        range_color=range_color,
        color_continuous_midpoint=np.mean(data_df["speed"]),
        color_continuous_scale=px.colors.diverging.RdBu_r[::-1],
        labels={"speed": "Speed"},
    )
    x, y = area.geoms[0].exterior.xy
    fig.add_trace(
        go.Scatter(
            x=np.array(x),
            y=np.array(y),
            mode="lines",
            line={"color": "grey"},
        )
    )
    for inner in area.geoms[0].interiors:
        xi, yi = zip(*inner.coords[:])
        fig.add_trace(
            go.Scatter(
                x=np.array(xi),
                y=np.array(yi),
                mode="lines",
                line={"color": "grey"},
            )
        )

    minx, miny, maxx, maxy = area.bounds
    fig.update_xaxes(range=[minx, maxx])
    fig.update_yaxes(range=[miny, maxy])
    fig.layout.updatemenus[0].buttons[0].args[1]["frame"]["duration"] = 10
    fig.layout.updatemenus[0].buttons[0].args[1]["transition"]["duration"] = 5
    fig.update_geos(
        projection_type="equirectangular", visible=True, resolution=110
    )
    fig.update_traces(marker={"line": {"width": 0.5, "color": "Gray"}})
    fig.update_geos(
        projection_type="equirectangular", visible=True, resolution=110
    )
    fig.update_coloraxes(colorbar=dict(title="Speed"))
    fig.update_layout(scene=dict(aspectmode="cube"), showlegend=False)
    fig.update_layout(yaxis=dict(scaleanchor="x", scaleratio=1))
    return fig


trajectory_data = read_trajectory_data(trajectory_file)
speed = pedpy.compute_individual_speed(traj_data=trajectory_data, frame_step=5)
speed = speed.merge(trajectory_data.data, on=["id", "frame"], how="left")
visualize(speed, GeometryCollection(walkable_area._polygon))

## References & Further Exploration

The operational model discussed in the Model section is based on the collision-free model. JuPedSim also incorporates another model known as GCFM. For more details on GCFM, refer to another notebook (TODO: Link to the GCFM notebook).

Our demonstration employed a straightforward journey with a singular exit. For a more intricate journey featuring multiple intermediate stops and waiting zones, see the upcoming section (TODO: Link to the advanced journey section).

While we designated a single parameter profile for agents in this example, it's feasible to define multiple parameter profiles. Learn how to alternate between these profiles in the subsequent section (TODO: Link to the profile-switching section).